# **Projeto - Redes Convolucionais**

### **Download dataset**

In [10]:
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from tensorflow.python.keras.callbacks import EarlyStopping

In [1]:
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

--2022-05-15 19:14:29--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/x-gzip]
Saving to: ‘images.tar.gz’

images.tar.gz       100%[===================>] 755.23M   111MB/s    in 6.9s    

2022-05-15 19:14:36 (109 MB/s) - ‘images.tar.gz’ saved [791918971/791918971]

--2022-05-15 19:14:36--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19173078 (18M) [application/x-gzip]
Saving to: ‘annotations.tar.gz’

annotations.tar.gz  100%[===================>]  18.28M  52.7MB/s    in 0.3s    

2022-05-15 19:14:37 (52.7 MB

In [2]:
!tar -xf images.tar.gz
!tar -xf annotations.tar.gz

### **Prepare data**

In [5]:
IMAGE_BASE_PATH = 'images'
SAMPLE_FILES = 'annotations/list.txt'
TRAIN_FILES = 'annotations/trainval.txt'
TEST_FILES = 'annotations/test.txt'

files = open(TRAIN_FILES)
train_files = []
train_labels = []
for txt_line in files:
    if txt_line[0] == '#':
        continue
    file_name, class_id, cat_dog_id, sub_class_id = txt_line.split(' ')
    train_files.append(file_name + '.jpg')
    train_labels.append(cat_dog_id)
files.close()

train_data_frame = pd.DataFrame({'filename': train_files, 'label': train_labels})

In [8]:
BATCH_SIZE = 64

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2, rescale=1/255)

train_dataset = train_datagen.flow_from_dataframe(dataframe=train_data_frame,
                                        directory=IMAGE_BASE_PATH,
                                        x_col='filename',
                                        y_col='label',
                                        target_size=(128, 128),
                                        class_mode='categorical',
                                        batch_size=BATCH_SIZE,
                                        subset='training',
                                        shuffle=True,
                                        seed=42)

val_dataset = train_datagen.flow_from_dataframe(dataframe=train_data_frame,
                                      directory=IMAGE_BASE_PATH,
                                      x_col='filename',
                                      y_col='label',
                                      target_size=(128, 128),
                                      class_mode='categorical',
                                      batch_size=BATCH_SIZE,
                                      subset='validation',
                                      shuffle=True,
                                      seed=42)

Found 2944 validated image filenames belonging to 2 classes.
Found 736 validated image filenames belonging to 2 classes.


### **Model - Xception**

In [11]:
#Pré treinada

rede = tf.keras.applications.Xception(
                  include_top=False,
                  weights="imagenet",
                  input_shape=(128, 128, 3),
                  classifier_activation="softmax"
              )

rede.trainable = False

83697664/83683744 [==============================] - 1s 0us/step


In [12]:
model = Sequential()
model.add(rede)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

In [13]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', min_delta = 0.001, patience = 5)

hist = model.fit_generator(train_dataset, steps_per_epoch = train_dataset.samples // 64, 
                           epochs = 20, validation_data = val_dataset, 
                           validation_steps = val_dataset.samples // 64, callbacks=[es])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1963: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
46/46 [==============================] - 27s 479ms/step - loss: 0.3243 - accuracy: 0.8862 - val_loss: 0.2077 - val_accuracy: 0.9261
Epoch 2/20
46/46 [==============================] - 20s 443ms/step - loss: 0.1317 - accuracy: 0.9555 - val_loss: 0.1712 - val_accuracy: 0.9375
Epoch 3/20
46/46 [==============================] - 25s 557ms/step - loss: 0.0671 - accuracy: 0.9745 - val_loss: 0.1452 - val_accuracy: 0.9432
Epoch 4/20
46/46 [==============================] - 20s 441ms/step - loss: 0.0340 - accuracy: 0.9888 - val_loss: 0.1401 - val_accuracy: 0.9574
Epoch 5/20
46/46 [==============================] - 23s 513ms/step - loss: 0.0193 - accuracy: 0.9939 - val_loss: 0.2270 - val_accuracy: 0.9347
Epoch 6/20
46/46 [==============================] - 27s 599ms/step - loss: 0.0163 - accuracy: 0.9946 - val_loss: 0.1944 - val_accuracy: 0.9531
Epoch 7/20
46/46 [==============================] - 20s 444ms/step - loss: 0.0332 - accuracy: 0.9891 - val_loss: 0.2817 - val_accuracy: 0.9375

In [14]:
#Descongelamento parcial

rede = tf.keras.applications.Xception(
                  include_top=False,
                  weights="imagenet",
                  input_shape=(128, 128, 3),
                  classifier_activation="softmax"
              )

print(len(rede.layers), len(rede.layers)//2)

for layer in rede.layers[:len(rede.layers)//2]:
   layer.trainable = False
for layer in rede.layers[len(rede.layers)//2:]:
   layer.trainable = True

132 66


In [15]:
model = Sequential()
model.add(rede)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

In [16]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', min_delta = 0.001, patience = 5)

hist = model.fit_generator(train_dataset, steps_per_epoch = train_dataset.samples // 64, 
                           epochs = 20, validation_data = val_dataset, 
                           validation_steps = val_dataset.samples // 64, callbacks=[es])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1963: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
46/46 [==============================] - 33s 577ms/step - loss: 0.3412 - accuracy: 0.8285 - val_loss: 97.3447 - val_accuracy: 0.6804
Epoch 2/20
46/46 [==============================] - 24s 524ms/step - loss: 0.1407 - accuracy: 0.9531 - val_loss: 3.9751 - val_accuracy: 0.8906
Epoch 3/20
46/46 [==============================] - 24s 522ms/step - loss: 0.0601 - accuracy: 0.9810 - val_loss: 1.0178 - val_accuracy: 0.9247
Epoch 4/20
46/46 [==============================] - 24s 521ms/step - loss: 0.0395 - accuracy: 0.9857 - val_loss: 0.4043 - val_accuracy: 0.9474
Epoch 5/20
46/46 [==============================] - 24s 516ms/step - loss: 0.0437 - accuracy: 0.9929 - val_loss: 0.3840 - val_accuracy: 0.9503
Epoch 6/20
46/46 [==============================] - 24s 523ms/step - loss: 0.0544 - accuracy: 0.9868 - val_loss: 0.8635 - val_accuracy: 0.8977
Epoch 7/20
46/46 [==============================] - 24s 521ms/step - loss: 0.0348 - accuracy: 0.9864 - val_loss: 0.3057 - val_accuracy: 0.951

In [17]:
#Descongelamento total

rede = tf.keras.applications.Xception(
                  include_top=False,
                  weights="imagenet",
                  input_shape=(128, 128, 3),
                  classifier_activation="softmax"
              )

rede.trainable = True

In [18]:
model = Sequential()
model.add(rede)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

In [19]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', min_delta = 0.001, patience = 5)

hist = model.fit_generator(train_dataset, steps_per_epoch = train_dataset.samples // 64, 
                           epochs = 20, validation_data = val_dataset, 
                           validation_steps = val_dataset.samples // 64, callbacks=[es])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1963: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
46/46 [==============================] - 49s 907ms/step - loss: 0.4108 - accuracy: 0.8342 - val_loss: 8.0407 - val_accuracy: 0.5994
Epoch 2/20
46/46 [==============================] - 41s 887ms/step - loss: 0.1735 - accuracy: 0.9402 - val_loss: 2.0026 - val_accuracy: 0.7429
Epoch 3/20
46/46 [==============================] - 41s 890ms/step - loss: 0.1015 - accuracy: 0.9643 - val_loss: 0.9376 - val_accuracy: 0.9261
Epoch 4/20
46/46 [==============================] - 41s 889ms/step - loss: 0.0648 - accuracy: 0.9779 - val_loss: 2.7778 - val_accuracy: 0.7571
Epoch 5/20
46/46 [==============================] - 41s 888ms/step - loss: 0.0325 - accuracy: 0.9864 - val_loss: 0.7950 - val_accuracy: 0.8764
Epoch 6/20
46/46 [==============================] - 41s 889ms/step - loss: 0.0337 - accuracy: 0.9908 - val_loss: 0.3544 - val_accuracy: 0.9347
Epoch 7/20
46/46 [==============================] - 41s 888ms/step - loss: 0.0501 - accuracy: 0.9834 - val_loss: 0.3032 - val_accuracy: 0.9176

**Observações:**

Uma das máximas acurácias entre os modelos desenvolvidos no projeto foi obtida utilizando a rede Xception e o overfitting desse modelo foi inferior aos anteriores. Ao realizar o descongelamento parcial e total a acurácia de validação foi um pouco menor, aumentando também o overfitting.